In [2]:
from automates.program_analysis.JSON2GroMEt.json2gromet import json_to_gromet

from automates.gromet.query import query
import gpt3 as gpt

ModuleNotFoundError: No module named 'automates'

The following assumes that you have already generated the json (e.g., by running the notebook script `py_src_to_grometFN_JSON.ipynb`)

In [2]:
gromet_fn_module = json_to_gromet("gromet/CHIME_SIR_while_loop--Gromet-FN-auto.json")

The following uses the simple query to collect all named output ports of a GrometFNModule.
`nops` will hold a `List` of `Tuples`, where each tuple has the following format:
```
(<name_of_output_port>,    # named output port == a variable in source code
 <literal_value>,          # IF a literal value has been assigned, otherwise None
 <source_code_reference>)  # metadatum for the source code location of the assignment
```

In [3]:
nops = query.collect_named_output_ports(gromet_fn_module)
print(nops[1])

('updated_growth_rate', None, {'code_file_reference_uid': '09a64930-e9e4-8ae5-f825-bdf7c1e729f9',
 'col_begin': 4,
 'col_end': 55,
 'line_begin': 4,
 'line_end': 4,
 'metadata_type': 'source_code_reference',
 'provenance': {'method': 'skema_code2fn_program_analysis',
                'timestamp': '2022-11-30 16:18:33.012974'}})


In [ ]:
MIRA_DKG_URL = 'http://34.230.33.149:8771'

def get_mira_dkg_term(term, attribs):
    res = requests.get(MIRA_DKG_URL + '/api/search', params={'q': term})
    term = [entity for entity in res.json() if entity['id'].startswith('askemo')][0]
    res = {attrib: term.get(attrib) for attrib in attribs if term.get(attrib) is not None}
    return res

In [ ]:
# Terms we want to find in MIRA and specific attributes we want to add to our local ontology
terms = ['population', 'doubling time', 'recovery time', 'infectious time']
attribs = ['description', 'synonyms', 'xrefs', 'suggested_unit', 'suggested_data_type',
           'physical_min', 'physical_max', 'typical_min', 'typical_max']

# The local ontology if filled up from the MIRA DKG
LOCAL_ONTOLOGY = {
    term: get_mira_dkg_term(term, attribs) for term in terms
    }

# We can also set further local / use-case specific constraints as needed
LOCAL_ONTOLOGY['population']['typical_min'] = 1000
LOCAL_ONTOLOGY['population']['typical_max'] = 40_000_000

LOCAL_ONTOLOGY

The following prints each name-output-port info tuple on a separate line.

In [1]:

targets = ['population', 'infectious time']
terms = ['population', 'doubling time', 'recovery time', 'infectious time']

parameters = set()
var_dict = {}
for nop in nops:
    if nop[1] is not None:
        parameters.add(nop[0])
        var_dict[nop[0]] = nop
#         print(type(nop[1].to_dict()))
# print((var_dict))
# print(len(parameters))
discoveredParameterConnections = gpt.match_gromet_targets(targets, list(parameters), var_dict, terms)
discoveredParameterConnections

NameError: name 'nops' is not defined

In [5]:
code = "CHIME_SIR_default_model.py"
targets = ['population', 'infectious time']
discoveredParameterConnections = []
try:
    discoveredParameterConnections = gpt.match_code_targets(targets, code, terms)
except OpenAIError as err:
    print("OpenAI connection error:", err)
    print("Using hard-coded connections")
    discoveredParameterConnections = [("infectious time", {"name": "grometSubObject"}, 14.0, 67),("population", {"name": "grometSubObject"}, 1000, 80)]

discoveredParameterConnections

1 (inv_contact_rate, 1.0)
2 (growth_rate, 0)
3 (growth_rate, 2.0)
4 (index, 0)
5 (p_idx, 0)
6 (d_idx, 0)
7 (i_day, 17.0)
8 (n_days, 20)
9 (N_p, 3)
10 (N_t, 121)
11 (infections_days, 14.0)
12 (relative_contact_rate, 0.05)
13 (gamma, 1.0)
14 (s_n, 1000)
15 (i_n, 1)
16 (r_n, 1)
17 (p_idx, 0)

[population, doubling time, recovery time, infectious time]
We have a list of variable names and initial values for lines of code in a Python program.The list has 17 elements
1 (inv_contact_rate, 1.0)
2 (growth_rate, 0)
3 (growth_rate, 2.0)
4 (index, 0)
5 (p_idx, 0)
6 (d_idx, 0)
7 (i_day, 17.0)
8 (n_days, 20)
9 (N_p, 3)
10 (N_t, 121)
11 (infections_days, 14.0)
12 (relative_contact_rate, 0.05)
13 (gamma, 1.0)
14 (s_n, 1000)
15 (i_n, 1)
16 (r_n, 1)
17 (p_idx, 0)

There is also an ontology which has 4 items:
ontology = [population, doubling time, recovery time, infectious time]
Some code elements have a matching element in the ontology. For example,
(s_n, 1000) matches 'population'
(relative_contact_rat

[('population', {'s_n': 'grometSubObject'}, 1000.0, 81),
 ('infectious time', {'infections_days': 'grometSubObject'}, 14.0, 68)]

In [7]:
import os

# The following should be edited to point to you own local path 
# to your local copy of the ASKEM-SKEMA google drive contents
ROOT = "/Users/claytonm/My Drive/ASKEM-SKEMA/data/"

EXP0_GROMET_JSON = os.path.join(ROOT, "gromet/examples/exp0/FN_0.1.4/exp0--Gromet-FN-auto.json")
EXP1_GROMET_JSON = os.path.join(ROOT, "gromet/examples/exp1/FN_0.1.4/exp1--Gromet-FN-auto.json")
EXP2_GROMET_JSON = os.path.join(ROOT, "gromet/examples/exp2/FN_0.1.4/exp2--Gromet-FN-auto.json")
CHIME_SIR_GROMET_JSON = os.path.join(ROOT, "epidemiology/CHIME/CHIME_SIR_model/gromet/FN_0.1.4/CHIME_SIR--Gromet-FN-auto.json")